In [ ]:
## Libraries
import os
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import TkinterDnD, DND_FILES
from PyPDF2 import PdfMerger


In [2]:

class PDFMergerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PDF Merger")
        self.root.geometry("600x400")

        # Lista para almacenar los archivos PDF seleccionados
        self.pdf_files = []

        # Crear la interfaz gráfica
        self.create_widgets()

    def create_widgets(self):
        # Panel de lista de archivos PDF
        self.listbox = tk.Listbox(self.root, selectmode=tk.SINGLE, width=80, height=15)
        self.listbox.pack(pady=10)

        # Botones para mover archivos hacia arriba o abajo
        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=5)

        btn_up = tk.Button(button_frame, text="Subir", command=self.move_up)
        btn_up.pack(side=tk.LEFT, padx=5)

        btn_down = tk.Button(button_frame, text="Bajar", command=self.move_down)
        btn_down.pack(side=tk.LEFT, padx=5)

        # Botón para eliminar archivos seleccionados
        btn_remove = tk.Button(button_frame, text="Eliminar", command=self.remove_selected)
        btn_remove.pack(side=tk.LEFT, padx=5)

        # Botón para fusionar los archivos PDF
        btn_merge = tk.Button(self.root, text="Fusionar PDFs", command=self.merge_pdfs)
        btn_merge.pack(pady=10)

        # Permitir arrastrar y soltar archivos en el Listbox
        self.listbox.drop_target_register(DND_FILES)
        self.listbox.dnd_bind('<<Drop>>', self.on_drop)

    def on_drop(self, event):
        # Obtener los archivos arrastrados
        dropped_files = event.data.split()
        for file in dropped_files:
            if file.lower().endswith('.pdf'):
                if file in self.pdf_files:
                    messagebox.showwarning("Advertencia", f"El archivo '{os.path.basename(file)}' ya está en la lista.")
                else:
                    self.pdf_files.append(file)
                    self.update_listbox()
            else:
                messagebox.showerror("Error", f"El archivo '{os.path.basename(file)}' no es un archivo PDF válido.")

    def update_listbox(self):
        # Actualizar el contenido del Listbox
        self.listbox.delete(0, tk.END)
        for file in self.pdf_files:
            self.listbox.insert(tk.END, os.path.basename(file))

    def move_up(self):
        # Mover el archivo seleccionado hacia arriba
        selected_index = self.listbox.curselection()
        if selected_index and selected_index[0] > 0:
            index = selected_index[0]
            self.pdf_files[index], self.pdf_files[index - 1] = self.pdf_files[index - 1], self.pdf_files[index]
            self.update_listbox()
            self.listbox.selection_set(index - 1)

    def move_down(self):
        # Mover el archivo seleccionado hacia abajo
        selected_index = self.listbox.curselection()
        if selected_index and selected_index[0] < len(self.pdf_files) - 1:
            index = selected_index[0]
            self.pdf_files[index], self.pdf_files[index + 1] = self.pdf_files[index + 1], self.pdf_files[index]
            self.update_listbox()
            self.listbox.selection_set(index + 1)

    def remove_selected(self):
        # Eliminar el archivo seleccionado de la lista
        selected_index = self.listbox.curselection()
        if selected_index:
            del self.pdf_files[selected_index[0]]
            self.update_listbox()

    def merge_pdfs(self):
        # Fusionar los archivos PDF seleccionados
        if not self.pdf_files:
            messagebox.showwarning("Advertencia", "No hay archivos PDF para fusionar.")
            return

        # Pedir al usuario el nombre del archivo de salida
        output_file = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF Files", "*.pdf")])
        if not output_file:
            return

        try:
            merger = PdfMerger()
            for pdf in self.pdf_files:
                merger.append(pdf)
            merger.write(output_file)
            merger.close()
            messagebox.showinfo("Éxito", f"Los PDFs han sido fusionados correctamente en:\n{output_file}")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo fusionar los PDFs:\n{str(e)}")

if __name__ == "__main__":
    root = TkinterDnD.Tk()
    app = PDFMergerApp(root)
    root.mainloop()

Puedes hacer un programa en python que encadene uno o más archivos .PDF en un único y nuevo pdf. Que dicho programa tenga una interfaz gráfica y que los pdf a encadenar puedan ser arrastrados a un panel de dicha interfaz grafica. Que los archivos se encadene en el orden que aparecen en dicho panel de arriba hacia abajo. Y que el orden de dichos pdf se pueda mover dentro del panel, o sea que tenga funcionalidad para mover archivos hacia arriba o hacia abajo dentro del  panel. Y además si agrego un archivo que ya está en la lista a encadenar aparezca un mensaje de Warning advirtiendo esa situación, pero que no me impida adicionarlo. Además que permita también eliminar archivos de la lista de panel

A continuación, te proporciono un programa en Python que cumple con los requisitos que has descrito. Este programa utiliza la biblioteca `tkinter` para la interfaz gráfica y `PyPDF2` para manipular archivos PDF. Además, se incluye funcionalidad para arrastrar y soltar archivos, reordenarlos, manejar advertencias de duplicados y eliminar archivos de la lista.

### Código del Programa

```python
import os
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import TkinterDnD, DND_FILES
from PyPDF2 import PdfMerger

class PDFMergerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PDF Merger")
        self.root.geometry("600x400")

        # Lista para almacenar los archivos PDF seleccionados
        self.pdf_files = []

        # Crear la interfaz gráfica
        self.create_widgets()

    def create_widgets(self):
        # Panel de lista de archivos PDF
        self.listbox = tk.Listbox(self.root, selectmode=tk.SINGLE, width=80, height=15)
        self.listbox.pack(pady=10)

        # Botones para mover archivos hacia arriba o abajo
        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=5)

        btn_up = tk.Button(button_frame, text="Subir", command=self.move_up)
        btn_up.pack(side=tk.LEFT, padx=5)

        btn_down = tk.Button(button_frame, text="Bajar", command=self.move_down)
        btn_down.pack(side=tk.LEFT, padx=5)

        # Botón para eliminar archivos seleccionados
        btn_remove = tk.Button(button_frame, text="Eliminar", command=self.remove_selected)
        btn_remove.pack(side=tk.LEFT, padx=5)

        # Botón para fusionar los archivos PDF
        btn_merge = tk.Button(self.root, text="Fusionar PDFs", command=self.merge_pdfs)
        btn_merge.pack(pady=10)

        # Permitir arrastrar y soltar archivos en el Listbox
        self.listbox.drop_target_register(DND_FILES)
        self.listbox.dnd_bind('<<Drop>>', self.on_drop)

    def on_drop(self, event):
        # Obtener los archivos arrastrados
        dropped_files = event.data.split()
        for file in dropped_files:
            if file.lower().endswith('.pdf'):
                if file in self.pdf_files:
                    messagebox.showwarning("Advertencia", f"El archivo '{os.path.basename(file)}' ya está en la lista.")
                else:
                    self.pdf_files.append(file)
                    self.update_listbox()
            else:
                messagebox.showerror("Error", f"El archivo '{os.path.basename(file)}' no es un archivo PDF válido.")

    def update_listbox(self):
        # Actualizar el contenido del Listbox
        self.listbox.delete(0, tk.END)
        for file in self.pdf_files:
            self.listbox.insert(tk.END, os.path.basename(file))

    def move_up(self):
        # Mover el archivo seleccionado hacia arriba
        selected_index = self.listbox.curselection()
        if selected_index and selected_index[0] > 0:
            index = selected_index[0]
            self.pdf_files[index], self.pdf_files[index - 1] = self.pdf_files[index - 1], self.pdf_files[index]
            self.update_listbox()
            self.listbox.selection_set(index - 1)

    def move_down(self):
        # Mover el archivo seleccionado hacia abajo
        selected_index = self.listbox.curselection()
        if selected_index and selected_index[0] < len(self.pdf_files) - 1:
            index = selected_index[0]
            self.pdf_files[index], self.pdf_files[index + 1] = self.pdf_files[index + 1], self.pdf_files[index]
            self.update_listbox()
            self.listbox.selection_set(index + 1)

    def remove_selected(self):
        # Eliminar el archivo seleccionado de la lista
        selected_index = self.listbox.curselection()
        if selected_index:
            del self.pdf_files[selected_index[0]]
            self.update_listbox()

    def merge_pdfs(self):
        # Fusionar los archivos PDF seleccionados
        if not self.pdf_files:
            messagebox.showwarning("Advertencia", "No hay archivos PDF para fusionar.")
            return

        # Pedir al usuario el nombre del archivo de salida
        output_file = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF Files", "*.pdf")])
        if not output_file:
            return

        try:
            merger = PdfMerger()
            for pdf in self.pdf_files:
                merger.append(pdf)
            merger.write(output_file)
            merger.close()
            messagebox.showinfo("Éxito", f"Los PDFs han sido fusionados correctamente en:\n{output_file}")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo fusionar los PDFs:\n{str(e)}")

if __name__ == "__main__":
    root = TkinterDnD.Tk()
    app = PDFMergerApp(root)
    root.mainloop()
```

### Requisitos Previos

1. **Instalación de Dependencias**:
   Asegúrate de tener instaladas las siguientes bibliotecas:
   ```bash
   pip install PyPDF2 tkinterdnd2
   ```

2. **Compatibilidad**:
   Este programa funciona en sistemas operativos como Windows, Linux y macOS. Sin embargo, `tkinterdnd2` puede requerir configuraciones adicionales en algunos sistemas.

### Funcionalidades del Programa

1. **Arrastrar y Soltar Archivos**:
   Puedes arrastrar archivos PDF al panel (Listbox) y se agregarán a la lista.

2. **Advertencia de Duplicados**:
   Si intentas agregar un archivo que ya está en la lista, aparecerá una advertencia, pero no se impedirá su adición.

3. **Reordenar Archivos**:
   Puedes mover los archivos hacia arriba o hacia abajo en la lista utilizando los botones "Subir" y "Bajar".

4. **Eliminar Archivos**:
   Selecciona un archivo en la lista y haz clic en "Eliminar" para quitarlo.

5. **Fusionar PDFs**:
   Haz clic en "Fusionar PDFs" para combinar todos los archivos en un único PDF. El programa te pedirá elegir el nombre y ubicación del archivo resultante.

### Ejecución

Guarda el código en un archivo `.py`, instala las dependencias necesarias y ejecuta el programa. La interfaz gráfica te permitirá realizar todas las operaciones descritas.

Si tienes alguna pregunta o necesitas más funcionalidades, no dudes en preguntar. ¡Espero que esto te sea útil!

Para convertir el código Python en un archivo ejecutable `.exe` para Windows 10, puedes usar herramientas como `PyInstaller` o `cx_Freeze`. A continuación, te explico cómo hacerlo con **PyInstaller**, que es una de las opciones más populares y fáciles de usar.

---

### **Opción 1: Usar PyInstaller**

#### **Paso 1: Instalar PyInstaller**
Abre una terminal (CMD o PowerShell) y ejecuta el siguiente comando para instalar `PyInstaller`:
```bash
pip install pyinstaller
```

#### **Paso 2: Preparar el Código**
Asegúrate de que tu script Python (`pdf_merger.py`) esté listo y funcione correctamente. Si usaste bibliotecas externas como `PyPDF2` y `tkinterdnd2`, asegúrate de que estén instaladas en tu entorno.

#### **Paso 3: Crear el Archivo .exe**
Navega al directorio donde está ubicado tu archivo Python y ejecuta el siguiente comando:
```bash
pyinstaller --onefile --windowed pdf_merger.py
```

- **`--onefile`**: Genera un solo archivo `.exe` en lugar de una carpeta con múltiples archivos.
- **`--windowed`**: Evita que se abra una ventana de terminal junto con la aplicación (útil para aplicaciones gráficas).

#### **Paso 4: Encontrar el Ejecutable**
Una vez que `PyInstaller` termine de procesar, el archivo `.exe` estará en la carpeta `dist` dentro del directorio de tu proyecto. Por ejemplo:
```
tu_proyecto/dist/pdf_merger.exe
```

#### **Paso 5: Probar el Ejecutable**
Haz doble clic en el archivo `.exe` para asegurarte de que funciona correctamente. Si hay algún problema, revisa los mensajes de error y ajusta el proceso según sea necesario.

---

### **Solución de Problemas Comunes con PyInstaller**

1. **Dependencias Externas**:
   Si usas bibliotecas como `tkinterdnd2`, puede ser necesario incluir manualmente algunos archivos adicionales. Por ejemplo:
   ```bash
   pyinstaller --onefile --windowed --add-data "ruta_a_tkinterdnd2;tkinterdnd2" pdf_merger.py
   ```

2. **Icono Personalizado**:
   Puedes agregar un ícono personalizado a tu ejecutable usando la opción `--icon`:
   ```bash
   pyinstaller --onefile --windowed --icon=mi_icono.ico pdf_merger.py
   ```

3. **Archivos Adicionales**:
   Si tu programa necesita archivos adicionales (como imágenes o datos), usa la opción `--add-data` para incluirlos. El formato es:
   ```
   --add-data "ruta_origen;ruta_destino"
   ```

---

### **Opción 2: Usar cx_Freeze**

`cx_Freeze` es otra herramienta popular para crear ejecutables. Aquí tienes un resumen de cómo usarla:

#### **Paso 1: Instalar cx_Freeze**
```bash
pip install cx_Freeze
```

#### **Paso 2: Crear un Script de Configuración**
Crea un archivo llamado `setup.py` en el mismo directorio que tu script principal (`pdf_merger.py`). Este archivo debe contener lo siguiente:
```python
from cx_Freeze import setup, Executable

setup(
    name="PDFMerger",
    version="1.0",
    description="Herramienta para fusionar PDFs",
    executables=[Executable("pdf_merger.py", base="Win32GUI")],  # Usa "Win32GUI" para aplicaciones gráficas
)
```

#### **Paso 3: Generar el Ejecutable**
Ejecuta el siguiente comando en la terminal:
```bash
python setup.py build
```

El archivo `.exe` se generará en la carpeta `build`.

---

### **Comparación entre PyInstaller y cx_Freeze**

| Característica           | PyInstaller                          | cx_Freeze                           |
|--------------------------|--------------------------------------|-------------------------------------|
| Facilidad de uso         | Muy fácil, ideal para principiantes | Requiere un archivo de configuración |
| Soporte para OneFile     | Sí                                  | No directamente (requiere configuración) |
| Personalización          | Opciones básicas son suficientes     | Más flexible pero más complejo       |
| Compatibilidad           | Amplia                              | Buena, pero menos popular            |

---

### **Recomendación Final**
Si buscas una solución rápida y sencilla, **PyInstaller** es la mejor opción. Es ampliamente utilizado, tiene una gran comunidad y es muy confiable. Sin embargo, si necesitas más control sobre el proceso de construcción, puedes optar por **cx_Freeze**.

¡Espero que esta guía te sea útil para crear tu archivo `.exe`! Si tienes alguna duda adicional, no dudes en preguntar. 😊